# Arquitetura Medallion
A Arquitetura Medallion é uma abordagem para construir um Data Warehouse ou Data Lake focada em qualidade, confiabilidade e acessibilidade dos dados. Ela organiza os dados em camadas com níveis crescentes de refinamento, estrutura e valor de negócio, similar a um processo de lapidação de um diamante bruto até uma joia polida e valiosa.

As camadas da Arquitetura Medallion são:

1. Bronze (Bruta):

* Objetivo: armazenar os dados brutos, exatamente como foram capturados das fontes originais, sem qualquer transformação ou limpeza.
* Formato: arquivos no formato original (CSV, JSON, XML, etc.), geralmente armazenados em um Data Lake (e.g., HDFS, S3).
* Características:
  * Dados brutos e não processados.
  * Schema-on-read (o esquema é inferido quando os dados são lidos).
  * Alta variedade e volume de dados.
  * Baixa latência de ingestão (dados são disponibilizados rapidamente).
  * Exemplo: logs de aplicação, feeds de dados de sensores, dados de redes sociais.

2. Silver (Aprimorada):

* Objetivo: limpar, transformar e enriquecer os dados brutos da camada Bronze.
* Formato: dados estruturados em formatos como Parquet ou ORC, armazenados em um Data Lake ou Data Warehouse.
* Características:
  * Dados limpos, consistentes e com maior qualidade.
  * Schema-on-write (o esquema é definido antes da escrita dos dados).
  * Dados desduplicados e com valores nulos tratados.
  * Adição de informações contextuais e enriquecimento dos dados.
  * Exemplo: dados de clientes com informações demográficas unificadas e padronizadas, dados de vendas com informações de produtos e promoções.
3. Gold (Refinada):

* Objetivo: criar datasets agregados e otimizados para análises de negócios e tomada de decisão.
* Formato: tabelas dimensionais e fatos, armazenadas em um Data Warehouse, Data Marts ou agregados pré-calculados para dashboards.
* Características:
  * Dados altamente estruturados e organizados para atender às necessidades específicas de negócio.
  * Dados históricos e dados atuais consolidados.
  * Dados otimizados para performance em consultas analíticas.
  * Alta qualidade e confiabilidade dos dados.
  * Exemplo: tabelas de dimensão de tempo, cliente e produto, tabelas de fatos de vendas, indicadores chave de performance (KPIs) pré-calculados.

Benefícios da Arquitetura Medallion:

* Escalabilidade e flexibilidade: permite lidar com grandes volumes e variedade de dados.
* Qualidade e confiabilidade dos dados: assegura a qualidade dos dados através de um processo incremental de refinamento.
* Agilidade: facilita a ingestão e processamento de novos dados.
* Reutilização de dados: permite que os mesmos dados brutos sejam utilizados para diferentes propósitos.
* Governança de dados: facilita a gestão e controle dos dados em cada camada.

![medallion-architecture](https://miro.medium.com/v2/resize:fit:1400/1*O4ey_K0ZbsESf8na7OirJg.jpeg)


# Download dos arquivos

Iremos baixar os arquivos de entrada:
- clients.csv
- vendas.csv

Os dados em `vendas.csv` são relativos a vendas realizadas por atacadistas e distribuidores.

## Dados de clientes

Iremos realizar o download de dados de clientes do link abaixo:

In [ ]:
!wget -O clients.csv https://www.dropbox.com/scl/fi/vd5hmlr7ghj2j5rx3w681/clients.csv?rlkey=rmcalhytfjm6nfklw7hhtykid&dl=1

**Se não funcionar o download acima, tente o link abaixo:**

---



In [ ]:
#!gdown https://drive.google.com/uc?id=1SQn8nCPhdFXFOe2wZ9wn1exTAIdgo2QU

## Dados de vendas

Iremos realizar o download dos dados de vendas presentes no arquivo `vendas.csv`:

In [ ]:
!wget -O vendas.csv https://www.dropbox.com/scl/fi/y6h3do8rp9fhovunvj36c/vendas.csv?rlkey=m4yl4h8vzfyg5fq8vyb2sbd2x&st=nz4dme6m&dl=1

**Se não funcionar o download acima, tente o link abaixo:**

In [ ]:
#!gdown https://drive.google.com/uc?id=1ubiLTdjEdy8C86MdkW1HRyPOFZl4irT1

# Analisando dados de vendas
Você está recebendo um conjunto de dados histórico de vendas de ERPs de várias empresas. Temos o histórico de vendas de várias empresas dentro do arquivo e, por isso, podemos ter períodos históricos diferentes de dados de vendas disponíveis.

## Arquivos
- **vendas.csv** - contém dados históricos de vendas até junho de 2022.
- **clients.csv** - dados dos clientes que compraram o produto.

## Campos do arquivo de vendas

- *client_id*: id do cliente.
- *items_count*: número de itens vendidos
- *list_price*: preço do produto no catálogo da empresa.
- *order_date*: data da venda.
- *order_id*: id do pedido. Cada pedido pode conter vários produtos vendidos dentro dele.
- *product_id*: id do produto vendido.
- *sale_price*: preço vendido ao cliente.
- *salesman_id*: id do vendedor.
- *supplier_id*: id do fornecedor do produto. Por exemplo, a indústria fabricando do produto.
- *company_id*: id da empresa. Temos dentro da base o histórico de vendas de várias empresas para clientes finais.
- *product*: nome do produto.
- *salesman*: nome do vendedor.
- *supplier*: nome do fornecedor.
- *client*: nome do cliente.


## Campos do arquivo clients.csv
- *client_id*: id do cliente.
- *cnae_id*: CNAE do cliente que está realizando a compra.
- *cod_city*: código da cidade no IBGE em que o cliente está localizado.
- *cod_tract*: código do setor censitário no IBGE em que o cliente está localizado.
- *cod_uf*: código da UF no IBGE em que o cliente está localizado.
- *city*: cidade do cliente.
- *state*: UF do cliente.
- *client*: nome do cliente.
- *company_id*: id da empresa. Temos dentro da base o histórico de vendas de várias empresas para clientes finais.


## Iniciando o PySpark

Esta célula de código instala o Spark no ambiente de execução Colab. Aqui está uma explicação passo a passo:

1. **`!apt-get install openjdk-11-jdk-headless -qq > /dev/null`**: este comando instala o OpenJDK 11 (versão headless, sem interface gráfica), que é um requisito para o Spark. O `-qq` suprime a saída e o `> /dev/null` redireciona a saída para o nada, tornando o processo mais silencioso.

2. **`!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz`**: Este comando baixa o arquivo compactado do Spark 3.5.2 (construído para o Hadoop 3) do site oficial do Apache Spark. O `-q` suprime a saída de download.

3. **`!tar xf spark-3.5.2-bin-hadoop3.tgz`**: Este comando extrai o arquivo compactado baixado do Spark, criando um diretório chamado `spark-3.5.2-bin-hadoop3`.

4. **`!pip -q install findspark`**: Este comando instala a biblioteca `findspark` usando `pip`. Findspark é uma biblioteca Python que torna mais fácil configurar o Spark em um ambiente Python, principalmente no Colab. Ela define as variáveis de ambiente necessárias para que o Spark funcione corretamente.

Após executar essas linhas, você terá o Spark instalado e pronto para ser usado em seu notebook Colab.

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip -q install findspark

Defina as variáveis de ambiente do Spark:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.2-bin-hadoop3"

O código a seguir garante que o Spark seja configurado corretamente e esteja pronto para uso em seu ambiente Python.

* **`findspark.init()`**: executa a função `init()` do módulo `findspark`. Esta função:
    * Localiza a instalação do Spark em seu sistema.
    * Configura as variáveis de ambiente necessárias para que o Python possa interagir com o Spark. Isso permite que o driver Python (seu código Python) se comunique com o executor Spark (o código que realmente processa os dados).


In [ ]:
import findspark
findspark.init()

Depois de executar a célula anterior, você poderá importar e usar as bibliotecas Spark como `pyspark.sql.SparkSession` para criar uma sessão Spark e começar a trabalhar com dados.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('Aula 1').master("local[*]").getOrCreate()

# Primeira etapa: carregar os arquivos

Nessa etapa você deve carregar os quatro arquivos abaixos, utilizando o **Spark**

**Dicas:**

- Separador dos arquivos é , (vírgula)
- Os arquivos possuem cabeçalho

In [ ]:
clients_df = spark.read.csv("clients.csv", header=True, inferSchema=True)

In [ ]:
clients_df.show()

In [ ]:
clients_df.printSchema()

In [ ]:
clients_schema = "city string, client_id string, cnae_id string, \
cod_city integer, cod_tract long, cod_uf integer, state string, client string, \
 company_id integer"
clients_df = spark.read.csv("clients.csv", header=True, schema=clients_schema, mode="DROPMALFORMED")

In [ ]:
vendas_df = spark.read.csv("vendas.csv",header=True)

In [ ]:
vendas_schema = "client_id string, items_count integer, list_price float, \
order_date date, order_id integer, product_id string, sale_price float, \
salesman_id string, supplier_id string, company_id integer, product string, \
salesman string, supplier string, client string"
vendas_df = spark.read.csv("vendas.csv",header=True, schema=vendas_schema)

In [ ]:
vendas_df.printSchema()

In [ ]:
clients_df.show()

In [ ]:
vendas_df.show(1)

## Spark Pandas

### O que é Spark Pandas?

Spark Pandas é uma biblioteca que fornece uma interface similar ao Pandas para trabalhar com dados em clusters Apache Spark. Isso significa que você pode usar as mesmas funções e métodos do Pandas, mas com a capacidade de processar datasets imensos distribuídos em vários nós.

### Por que usar Spark Pandas?

Para cientistas de dados, o Spark Pandas oferece diversas vantagens:

- Escalabilidade: processa conjuntos de dados enormes com rapidez e eficiência, utilizando a computação distribuída do Spark.
- Familiaridade: permite utilizar a linguagem e as funções do Pandas, que você já conhece, para análise de dados em grande escala.
- Performance: aproveita as otimizações do Spark para acelerar tarefas como leitura, transformação e agregação de dados.
- Integração: funciona perfeitamente com outros componentes do ecossistema Spark, como Spark SQL e MLlib.

Agora vamos fazer a mesma operação de leitura de dados de clientes e vendas com **Spark Pandas** (vide [documentação](https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/index.html)).

Os clientes vamos armazenar no Dataframe `clients_pdf` e os dados de vendas em `vendas_pdf`. A leitura do csv é semelhante ao Pandas com o método `read_csv`.

In [ ]:
import pyspark.pandas as ps

In [ ]:
clients_pdf = ps.read_csv('clients.csv', names=clients_schema, header=0)

In [ ]:
clients_pdf.dtypes

In [ ]:
vendas_pdf = ps.read_csv("vendas.csv", names=vendas_schema, header=0)

In [ ]:
vendas_pdf.head(5)

# Consultas nas bases de dados

In [ ]:
clients_df.createOrReplaceTempView("clients")

In [ ]:
sql = """
select state, client_id
from clients
limit 10
"""

In [ ]:
spark.sql(sql).show()

In [ ]:
sql = """
select state, count(*) as contagem
from clients
where cnae_id is not null
group by state
order by contagem asc
limit 10
"""

In [ ]:
spark.sql(sql).show()

In [ ]:
clients_pdf[ clients_pdf.cnae_id.isnull() == False]['state'].value_counts().sort_values(
    ascending=False).head(10).to_frame()


In [ ]:
clients_df.filter("cnae_id is not null")\
.groupBy("state").count()\
.sort(desc("count"))\
.limit(10) \
.show()

In [ ]:
clients_pdf[clients_pdf['state'] == 'GO'].head(3)

In [ ]:
clients_df.filter(clients_df['state'] == 'GO').show(3)

# Camadas da Arquitetura Medallion

## Bronze (Dados Brutos):

* Carregar os arquivos CSV vendas.csv e clientes.csv como DataFrames Spark.
* Criar tabelas brutas na camada Bronze, armazenando os dados brutos sem alterações.

## Silver (Dados Limpos e Enriquecidos):

* Ler os dados das tabelas da camada Bronze.
* Limpeza de dados:
  * Remover duplicatas.
  * Tratar valores ausentes, utilizando estratégias adequadas para cada coluna (ex: preenchimento com valores padrão, médias, etc.).
  * Converter tipos de dados para os formatos corretos (ex: datas, números).
* Enriquecimento de dados:
  * Juntar as tabelas vendas e clientes utilizando a chave client_id.
  * Criar novas colunas com informações relevantes, como:
  `valor_total`: multiplicar `items_count` por `sale_price`.
  * Ano da venda (extraído da coluna `order_date`).
  * Mês da venda (extraído da coluna `order_date`).
  * Salvar os dados enriquecidos e limpos como tabelas na camada Silver.

## Gold (Dados Agregados):

* Ler os dados da camada Silver.
* Criar agregações para diferentes níveis de análise:
  * Total de vendas por dia.
  * Total de vendas por mês.
  * Total de vendas por cliente.
  * Total de vendas por produto.
  * Total de vendas por vendedor.

Armazenar os dados agregados em tabelas na camada Gold, otimizadas para consultas analíticas.

# Junção da base de dados de clientes e vendas

In [ ]:
joined_df = vendas_df.join(clients_df, "client_id", "left")

In [ ]:
vendas_df.show(1)

In [ ]:
joined_df.show(1)

In [ ]:
joined_pdf = vendas_pdf.merge(clients_pdf, on='client_id', how='left')

In [ ]:
len(joined_pdf)